# Hypothesis testing: aka answering YES/NO questions

So far we have covered quite some stuff like why the CLT is so important, what the standard error is, what is a confidence interval and the formulae for sound A/B testing. Now, effectively, at work some suit will show up and will ask you a question. The question will not be in the sorts of *"can you tell me the confidence interval of this test campaign?"*. 

It will be more basic, it will be more human and less nerdy. It will in the fashion of **"does this work or not?"**. 

And your answer has to be basically **YEP** or **NOPE**. 

After that, the suit -if clever- might frown and ask you *"right, but how sure are you?". And your answer has to be **"I am 95% sure"**.

After that, most possibly the suit smiles (the result is coincident with her intentions) or frowns even more and asks you to go ahead and re-check your math since this is not what she was expecting. In any case, that's not your problem anymore, **as long as your math are right**. And this is why it is important to follow this well. 

## The HP jargon

First things first, when you read a statistics book they will start putting names and conventions that are well known for the master jedis of stats but that are tricky and confusing for the padawans. Here's a summary for this jargon:

### Null Hypothesis (H0)
This is the *status quo*. That means that the default action is the happy choice. *Just do nothing*.

Generally H0 takes the form of 

$$ H_0 = \mu $$

### Alternative Hypothesis (H1)
That's when you think you need to update or do something since what your current assumption does not sound right anymore. 

Generally H1 takes the form of 

$$ H_1 > \mu, H_1 < \mu, H_1 \ne \mu $$

### Type I error
This is when you feck up because you decide you should update and change your mind but you actually shouldn't (rejecting  H0). Mnemotechnic: basically it's the same as leaving your wife for a younger bimbo just to realize afterwards that it wasn't worth it.

We normally call $\alpha$ the probability of screwing up this way:

$$\alpha = P(TypeI) $$

Just to be clear, we want $\alpha$ to be small, e.g. 5% for a 95% confidence.


### Type II error
This is when you feck up because you didn' update your assumptions but you should have (rejecting H1). Mnemotechnic: it's the same as not changing your very old car because you think it will last forever, until it leaves you stranded in the middle of nowhere.




|       TRUTH       |      DECIDE       |    what it is         |
|-------------------|-------------------|-----------------------|
|         H0        |        H0         | correctly accept H0   |
|         H0        |        H1         |     Type I error      |
|         H1        |        H1         | correctly reject H0   |
|         H1        |        H0         |     Type II error     |


### Test Statistic
This is a metric we will compute and that we will use to check whether we should or not reject H0. It takes the form:

$$ TS ={ {\bar X - \mu_0} \over {S \over \sqrt{n}}} $$

Basically what you want to do is to place the Test Stat on the abscissa of the distribution of your error (normal, or t-student). Then you plot the quantile at 95% (or whatever) coincidence aka $Z_{1-\alpha}$ and you compare then.

* If TS > Z --> reject H0.

Just do the normal maths, it basically means that your sample mean falls above (or below) the confidence interval and therefore you should do something about it.

* If TS < Z --> accept H0.

**Example:** Let's recover the example from before, and let's start with a null hypothesis that it is correct. In this case we should not update and we should not reject (and accept H0). Otherwise we are making a Type I error.

In [11]:
import numpy as np
import scipy.stats

p = 0.55
conf_alpha = 0.95
H0 = 0.55
for sample_size in [5, 7, 100, 1000, 10000]:
    sample = np.random.binomial(1, p, size=sample_size)
    sample_mean, sample_SE = np.mean(sample), np.std(sample)/np.sqrt(sample_size)
    Z_alpha = scipy.stats.t.ppf(conf_alpha, df=sample_size-1)
    TS = abs((sample_mean - H0)/sample_SE)
    decide = "Rejecting H0" if TS > Z_alpha else "Accepting H0"
    print("TS is {:.4f}, Z at {}% confidence is {:.4f}, with {} samples: {}".format(TS, 100*conf_alpha, Z_alpha, sample_size, decide))

TS is -1.9566, Z at 95.0% confidence is 2.1318, with 5 samples: Accepting H0
TS is 0.1146, Z at 95.0% confidence is 1.9432, with 7 samples: Accepting H0
TS is 0.0000, Z at 95.0% confidence is 1.6604, with 100 samples: Accepting H0
TS is -0.6345, Z at 95.0% confidence is 1.6464, with 1000 samples: Accepting H0
TS is -0.0201, Z at 95.0% confidence is 1.6450, with 10000 samples: Accepting H0


Now, let's try to test for another Null Hypothesis, we should now reject this or otherwise we would be making a Type II error.

Let's try to see if the probability is 0.8, while it is actually 0.55. Basically our Null Hypothesis then is:

$$H0 : \mu = 0.8$$

In [14]:
import numpy as np
import scipy.stats

p = 0.55
conf_alpha = 0.95
H0 = 0.8
for sample_size in [5, 7, 100, 1000, 10000]:
    sample = np.random.binomial(1, p, size=sample_size)
    sample_mean, sample_SE = np.mean(sample), np.std(sample)/np.sqrt(sample_size)
    Z_alpha = scipy.stats.t.ppf(conf_alpha, df=sample_size-1)
    TS = abs((sample_mean - H0)/sample_SE)
    decide = "Rejecting H0" if TS > Z_alpha else "Accepting H0"
    print("TS is {:.4f}, Z at {}% confidence is {:.4f}, with {} samples: {}".format(TS, 100*conf_alpha, Z_alpha, sample_size, decide))

TS is 1.8257, Z at 95.0% confidence is 2.1318, with 5 samples: Accepting H0
TS is 0.4320, Z at 95.0% confidence is 1.9432, with 7 samples: Accepting H0
TS is 4.6458, Z at 95.0% confidence is 1.6604, with 100 samples: Rejecting H0
TS is 15.7704, Z at 95.0% confidence is 1.6464, with 1000 samples: Rejecting H0
TS is 48.4253, Z at 95.0% confidence is 1.6450, with 10000 samples: Rejecting H0


You should have noticed two important things here:

* That the number of samples is very important since it drives the whole Standard Error. With vey little samples we just chicken out and we decide to stay as we are. The point is, we are not sure. That's why I prefer always to compute the confidence interval and show it as well. 

* The Test Stat I computed here includes an absolute term. I didn't lie to you, but think about this specific test, we were looking for an absolute difference of the means (is the mean 0.8 or not), it was not about whether the mean was below 0.8 (we would not reject H0, in this case).

## p-values

p-values are just another angle to the same thing. Basically it follows this logic:

* Assume that H0 is true.
* What is the probability of observing the data I have seen if H0 was true?

Well, that's the p-value.


If the p-value is small then it could be due to two things:

* H0 is false and we should reject it.
* We just have been very unlucky and we have observed a very rare thing, an outlier.

So, if $p-value < \alpha $ then you should reject H0.

## Statistics IRL

In real life, you will not be using all the quantiles and sample errors yourself. You will instead throw the samples to a library - which is 100% trustable - and just inspect the results. Here are some examples using python's scipy library, picking from the examples before. 

In [25]:
import numpy as np
import scipy.stats

p = 0.55
H0 = 0.55
alpha = 5/100

def run_ttest_binomial(p, H0, alpha):
    for sample_size in [5, 7, 100, 1000, 10000]:
        sample = np.random.binomial(1, p, size=sample_size)
        res = scipy.stats.ttest_1samp(sample, H0)
        decide = "Accept H0" if res.pvalue > alpha else "Reject H0"
        print("Sample size {}: {}, then {}, ".format(sample_size, res, decide))
        
run_ttest_binomial(p, H0, alpha)

Sample size 5: Ttest_1sampResult(statistic=-0.6123724356957946, pvalue=0.5733922538253555), then Accept H0, 
Sample size 7: Ttest_1sampResult(statistic=0.8907861696277058, pvalue=0.4073422632884692), then Accept H0, 
Sample size 100: Ttest_1sampResult(statistic=-2.2049052457775016, pvalue=0.029775735011297984), then Reject H0, 
Sample size 1000: Ttest_1sampResult(statistic=1.212667048593489, pvalue=0.22554382893766717), then Accept H0, 
Sample size 10000: Ttest_1sampResult(statistic=-0.6427899218817335, pvalue=0.5203751622094448), then Accept H0, 


In [26]:
H0 = 0.8
run_ttest_binomial(p, H0, alpha)

Sample size 5: Ttest_1sampResult(statistic=0.0, pvalue=1.0), then Accept H0, 
Sample size 7: Ttest_1sampResult(statistic=-1.1313708498984765, pvalue=0.3010822574245597), then Accept H0, 
Sample size 100: Ttest_1sampResult(statistic=-5.3826296292996405, pvalue=4.938387543587381e-07), then Reject H0, 
Sample size 1000: Ttest_1sampResult(statistic=-15.943439658595835, pvalue=3.7130488705693686e-51), then Reject H0, 
Sample size 10000: Ttest_1sampResult(statistic=-49.60122697572499, pvalue=0.0), then Reject H0, 


Let's try now to see if two samples have the same average, we can also do that with the same library